# Synchronize the opengrid data to your computer.  

There are two different solutions.  

1. The first approach is based on the **tmpo data format** and uses a local database. This approach is more elegant, but does not cover historic data from before 23rd of October 2014.
2. The second is based on **csv files** that are stored on the opengrid webserver.  These csv files are extracted from the flukso webserver and stored as a zip file per day.  This script will only download missing files and convert all the data into a single csv per sensor. This approach covers all possible opengrid data as for now.


In [ ]:
import os, sys
import inspect

# Obtain path of the opengrid codebase and import opengrid libraries
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))
from opengrid.library import fluksoapi
from opengrid.library import config
c = config.Config()

# for tmpo, only use user-speficied path if a correct path is provided
try:
    if os.path.exists(c.get('tmpo', 'data')):
        path_to_tmpo_data = c.get('tmpo', 'data')
except:
    path_to_tmpo_data = None

## TMPO-based synchronization

In [ ]:
from opengrid.library import houseprint
sys.path.append(c.get('tmpo', 'folder'))
import tmpo

Run the cell below if you want to obtain the very last houseprint information

In [ ]:
%run cache_anonymous_houseprint.py

Create a tmpo session and load the houseprint

In [ ]:
tmpos = tmpo.Session(path_to_tmpo_data)
tmpos.debug = True
hp = houseprint.load_houseprint_from_file('hp_anonymous.pkl')

Make sure all known sensors of the houseprint are added to the tmpo database.  Then synchronize all data up to now.

In [ ]:
tmpos = fluksoapi.update_tmpo(tmposession = tmpos, hp=hp)

In [ ]:
tmpos.sync()

## CSV-based synchronization

In [ ]:
# path to data is stored in opengrid.cfg.
# for syncing with csv
path_to_csv_data = c.get('data', 'folder') 
# This synchronization can take a while...
fluksoapi.synchronize(path_to_csv_data)
# see the other notebooks on how to import the data and start analysing.